<a href="https://colab.research.google.com/github/Aliminatorx/MobileNet-Pascal-VOC2012/blob/main/MobileNet_GCS2208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Importing other libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow for COLAB
#from cv2 import imshow as cv2_imshow # <--- use this for regular use.
import os
import re
import random
import time
import pickle

from skimage import io
import shutil
from tqdm import tqdm
import tarfile
import requests
import urllib.request
import xml.etree.ElementTree as ET
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
class VOCDataset():
  def __init__(self):
    self.train_dir=None
    self.test_dir=None
    self.trainDataLink=None
    self.testDataLink=None

    self.common_init()

  def common_init(self):
    # init that must be shared among all subclasses of this method
    self.label_type=['none','aeroplane',"Bicycle",'bird',"Boat","Bottle","Bus","Car","Cat","Chair",'cow',"Diningtable","Dog","Horse","Motorbike",'person', "Pottedplant",'sheep',"Sofa","Train","TVmonitor"]
    self.convert_id=['background','Aeroplane',"Bicycle",'Bird',"Boat","Bottle","Bus","Car","Cat","Chair",'Cow',"Dining table","Dog","Horse","Motorbike",'Person', "Potted plant",'Sheep',"Sofa","Train","TV/monitor"]
    self.convert_labels={}
    for idx, x in enumerate(self.label_type):
      self.convert_labels[x.lower()]=idx

    self.num_classes=len(self.label_type) # 20 + 1(none)

  def download_dataset(self, validation_size=5000):
    # download voc train dataset
    print('[*] Downloading dataset...')
    print(self.trainDataLink)
    urllib.request.urlretrieve(self.trainDataLink, 'voctrain.tar')

    print('[*] Extracting dataset...')
    tar = tarfile.open('voctrain.tar', "r:")
    tar.extractall('/content/VOCtrain')
    tar.close()
    os.remove('/content/voctrain.tar')

    if self.testDataLink is None: 
      # move 5K images to validation set
      print('[*] Moving validation data...')
      ensure_dir(self.test_dir+'/Annotations/')
      ensure_dir(self.test_dir+'/JPEGImages/')

      random.seed(42)
      val_images = random.sample(sorted(os.listdir(self.train_dir + '/JPEGImages')), validation_size)

      for path in val_images:
        img_name = path.split('/')[-1].split('.')[0]
        # move image
        os.rename(self.train_dir+'/JPEGImages/'+img_name+'.jpg', self.test_dir+'/JPEGImages/'+img_name+'.jpg')
        # move annotation
        os.rename(self.train_dir+'/Annotations/'+img_name+'.xml', self.test_dir+'/Annotations/'+img_name+'.xml')
    else: 
      # Load from val data
      print('[*] Downloading validation dataset...')
      urllib.request.urlretrieve(self.testDataLink, 'voctest.tar')

      print('[*] Extracting validation dataset...')
      tar = tarfile.open('voctest.tar', "r:")
      tar.extractall('/content/VOCtest')
      tar.close()
      os.remove('/content/voctest.tar')

  def read_xml(self, xml_path): 
    object_list=[]

    tree = ET.parse(open(xml_path, 'r'))
    root=tree.getroot()
  
    objects = root.findall("object")
    for _object in objects:
      name = _object.find("name").text
      bndbox = _object.find("bndbox")
      xmin = int(bndbox.find("xmin").text)
      ymin = int(bndbox.find("ymin").text)
      xmax = int(bndbox.find("xmax").text)
      ymax = int(bndbox.find("ymax").text)
      class_name = _object.find('name').text
      object_list.append({'x1':xmin, 'x2':xmax, 'y1':ymin, 'y2':ymax, 'class': self.convert_labels[class_name]})

    return object_list
class VOC2007(VOCDataset):
  def __init__(self):
    self.train_dir='/content/VOCtrain/VOCdevkit/VOC2007'
    self.test_dir='/content/VOCtest/VOCdevkit/VOC2007'
    self.trainDataLink='http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'
    self.testDataLink='http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar'
    self.common_init()#mandatory
    
class VOC2012(VOCDataset):
  def __init__(self):
    self.train_dir='/content/VOCtrain/VOCdevkit/VOC2012'
    self.test_dir='/content/VOCtest/VOCdevkit/VOC2012'
    # original site goes down frequently, so we use a link to the clone alternatively
    # self.trainDataLink='http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar' 
    self.trainDataLink = 'http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar'
    self.testDataLink=None
    self.common_init()#mandatory

In [10]:
voc_dataset=VOC2012()
voc_dataset.download_dataset()
val_datalen=len(os.listdir(voc_dataset.test_dir+'/Annotations'))
train_datalen=len(os.listdir(voc_dataset.train_dir+'/Annotations'))
print(train_datalen, val_datalen) # 17,125 total

[*] Downloading dataset...
http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
[*] Extracting dataset...
[*] Moving validation data...
12125 5000


In [11]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/VOCtrain/VOCdevkit/VOC2012",target_size=(227,227))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/VOCtest/VOCdevkit/VOC2012", target_size=(227,227))

Found 17951 images belonging to 5 classes.
Found 5000 images belonging to 2 classes.


In [12]:
IMAGE_SETS_PATH = '/content/VOCtrain/VOCdevkit/VOC2012/ImageSets/'
IMAGE_SETS_MAIN_PATH = '/content/VOCtrain/VOCdevkit/VOC2012/ImageSets/Main/'
JPEG_IMAGES_PATH = '/content/VOCtrain/VOCdevkit/VOC2012/JPEGImages/'

In [13]:
imageSets_list = os.listdir(IMAGE_SETS_PATH + 'Main')

In [14]:
train_file_list = []
val_file_list = []

for file_name in imageSets_list:
    if file_name.find('_train.txt') != -1:
        train_file_list.append(file_name)
    elif file_name.find('_val.txt') != -1:
        val_file_list.append(file_name)

In [15]:
import pandas as pd

In [16]:
train_dataframe = pd.DataFrame(columns=('file_name', 'class'))
val_dataframe = pd.DataFrame(columns=('file_name', 'class'))

kind = 0
for train_file_name in train_file_list:
    f = open(IMAGE_SETS_MAIN_PATH + train_file_name)
    row_list = f.readlines()
    for pair in row_list:
        target = pair.split()
        if target[1] == '1':
            train_dataframe = train_dataframe.append(
                [{'file_name': JPEG_IMAGES_PATH + target[0] + '.jpg',
                  'class': train_file_name[:-10]}],
                ignore_index=True
            )
    kind += 1
    f.close()

kind = 0
for val_file_name in val_file_list:
    f = open(IMAGE_SETS_MAIN_PATH + val_file_name)
    row_list = f.readlines()
    for pair in row_list:
        target = pair.split()
        if target[1] == '1':
            val_dataframe = val_dataframe.append(
                [{'file_name': JPEG_IMAGES_PATH + target[0] + '.jpg',
                  'class': val_file_name[:-8]}],
                ignore_index=True
            )
    kind += 1
    f.close()

In [17]:
train_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet.preprocess_input,
    validation_split=0.1
)

test_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet.preprocess_input
)

In [18]:
train_sets = train_generator.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col='file_name',
    y_col='class',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    subset='training',
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_sets = train_generator.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col='file_name',
    y_col='class',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    subset='validation'
)

test_sets = test_generator.flow_from_dataframe(
    dataframe=val_dataframe,
    x_col='file_name',
    y_col='class',
    target_size=(300, 300),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 5261 validated image filenames belonging to 20 classes.
Found 584 validated image filenames belonging to 20 classes.
Found 5964 validated image filenames belonging to 20 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2486 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2387 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [19]:
#Build MobileNet Model
original_model = keras.applications.mobilenet.MobileNet(
    input_shape=(300, 300, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

inputs = original_model.input

x = keras.layers.Dense(512, activation='relu')(original_model.output)
x = keras.layers.Dense(256, activation='relu')(x)

outputs = keras.layers.Dense(20, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss='mse', 
              optimizer='adam', 
              metrics=['accuracy'])

17235968/17225924 [==============================] - 0s 0us/step


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("mobilenet_model.h5",
                                                        monitor='val_loss',
                                                        verbose=1,
                                                        save_best_only=True,
                                                        mode='min')

history = model.fit(train_sets, 
                    epochs =20, 
                    verbose=1,
                    validation_data=val_sets,
                    callbacks=[checkpoint])



Epoch 1/20
165/165 [==============================] - ETA: 0s - loss: 0.0387 - accuracy: 0.4085 
Epoch 1: val_loss improved from inf to 0.06593, saving model to densenet_model.h5
165/165 [==============================] - 1786s 11s/step - loss: 0.0387 - accuracy: 0.4085 - val_loss: 0.0659 - val_accuracy: 0.0993
Epoch 2/20
165/165 [==============================] - ETA: 0s - loss: 0.0345 - accuracy: 0.4874 
Epoch 2: val_loss did not improve from 0.06593
165/165 [==============================] - 1760s 11s/step - loss: 0.0345 - accuracy: 0.4874 - val_loss: 0.0717 - val_accuracy: 0.0103
Epoch 3/20
165/165 [==============================] - ETA: 0s - loss: 0.0320 - accuracy: 0.5280 
Epoch 3: val_loss improved from 0.06593 to 0.06508, saving model to densenet_model.h5
165/165 [==============================] - 1782s 11s/step - loss: 0.0320 - accuracy: 0.5280 - val_loss: 0.0651 - val_accuracy: 0.0685
Epoch 4/20
165/165 [==============================] - ETA: 0s - loss: 0.0307 - accuracy: 0.5

In [ ]:
model.evaluate(test_sets, verbose=1)

187/187 [==============================] - 357s 2s/step - loss: 0.0366 - accuracy: 0.4930


[0.03656448423862457, 0.49295774102211]

In [ ]:
model.save('VOC_MobileNet.h5')